In [51]:
# Para llevar un registro de los cambios
%pip install -q comet_ml
import comet_ml
comet_ml.init(project_name='Tarea 03')

In [52]:
# Librerías
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint

In [53]:
experiment = comet_ml.Experiment(
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True,
)
experiment.set_name("Prueba 03.4")

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-03/4d46973aed3641daaf5c38a9134319ce
COMET INFO:   Others:
COMET INFO:     Name : Prueba 03.4
COMET INFO:   Parameters:
COMET INFO:     batch_size    : 15
COMET INFO:     epochs        : 30
COMET INFO:     learning_rate : 0.01
COMET INFO:     loss          : categorical_crossentropy
COMET INFO:     optimizer     : adam
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: As yo

In [54]:
# Carga el dataset de MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [55]:
# Convierte y normaliza los datos de entrada
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [56]:
# to_categorical convierte las etiquetas (train, test) en matrices one-hot
num_classes = 10
y_trainc = keras.utils.to_categorical(y_train, num_classes)
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [57]:
  # Diccionario con los parámetros que se registrarán en comet
  parameters = {
      'epochs' : 30,                                                            # 10 épocas menos
      'batch_size' : 15,
      'learning_rate' : 0.010,
      'loss' : 'categorical_crossentropy',
      'optimizer' : 'adam'
  }
  experiment.log_parameters(parameters)

In [58]:
# Creación del modelo
model = Sequential()
# Es como una "primera capa" de tres capas
model.add(Input(shape = (28,28))) # Matrices de entrada de 28x28 (imáegenes)
model.add(Flatten()) # Convierte las matrices de 28x28 en matrices de 784x1
model.add(Dense(784, activation = 'sigmoid'))                                   # ya estaba
model.add(Dropout(0.5))                                                         # ya estaba
model.add(Dense(500, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(500, activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(200, activation = 'elu'))
model.add(Dense(100, activation = 'relu')) # Segunda capa                       # cambio de 30 a 100 neuronas
model.add(Dense(10, activation = 'softmax')) # Tercera capa

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 784)               0         
                                                                 
 dense_15 (Dense)            (None, 784)               615440    
                                                                 
 dropout_7 (Dropout)         (None, 784)               0         
                                                                 
 dense_16 (Dense)            (None, 500)               392500    
                                                                 
 dropout_8 (Dropout)         (None, 500)               0         
                                                                 
 dense_17 (Dense)            (None, 500)               250500    
                                                                 
 dropout_9 (Dropout)         (None, 500)              

In [59]:
# Checkpoint que guarda el mejor modelo (cuando la función de costo alcanza un mínimo)
checkpoint = ModelCheckpoint(filepath = 'best_model.hdf5', monitor = 'val_loss', verbose = 1, save_best_only = True, mode = 'min')

In [60]:
# optimizer = RMSprop(learning_rate = 0.0005)
optimizer = 'adam'
model.compile(loss = parameters['loss'], optimizer = optimizer, metrics = ['accuracy'])
model.fit(x_train, y_trainc, batch_size = parameters['batch_size'], epochs = parameters['epochs'], verbose = 1, validation_data = (x_test, y_testc), callbacks = [checkpoint])

Epoch 1/30
3999/4000 [============================>.] - ETA: 0s - loss: 0.4358 - accuracy: 0.8603
Epoch 1: val_loss improved from inf to 0.16606, saving model to best_model.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4000/4000 [==============================] - 72s 18ms/step - loss: 0.4357 - accuracy: 0.8604 - val_loss: 0.1661 - val_accuracy: 0.9530
Epoch 2/30
4000/4000 [==============================] - ETA: 0s - loss: 0.2039 - accuracy: 0.9418
Epoch 2: val_loss improved from 0.16606 to 0.12246, saving model to best_model.hdf5
4000/4000 [==============================] - 66s 17ms/step - loss: 0.2039 - accuracy: 0.9418 - val_loss: 0.1225 - val_accuracy: 0.9660
Epoch 3/30
4000/4000 [==============================] - ETA: 0s - loss: 0.1622 - accuracy: 0.9545
Epoch 3: val_loss improved from 0.12246 to 0.10829, saving model to best_model.hdf5
4000/4000 [==============================] - 64s 16ms/step - loss: 0.1622 - accuracy: 0.9545 - val_loss: 0.1083 - val_accuracy: 0.9690
Epoch 4/30
3998/4000 [============================>.] - ETA: 0s - loss: 0.1366 - accuracy: 0.9623
Epoch 4: val_loss improved from 0.10829 to 0.10553, saving model to best_model.hdf5
4000/4000 [==============================] - 66s 

In [61]:
experiment.log_model('MNIST1', 'best_model.hdf5')
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/gurumasterahb/tarea-03/1dc758845865416ba0a43984586592dc
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [30]                  : (0.8603833317756653, 0.98826664686203)
COMET INFO:     batch_accuracy [12000]         : (0.0, 1.0)
COMET INFO:     batch_loss [12000]             : (7.40229879738763e-05, 2.5300133228302)
COMET INFO:     epoch_duration [30]            : (63.97844239799997, 73.10252708100052)
COMET INFO:     loss [30]                      : (0.05506934970617294, 0.4357285499572754)
COMET INFO:     val_accuracy [30]              : (0.953000009059906, 0.9843999743461609)
COMET INFO:     val_loss [30]    